Initialize Cohere API Key, Game & Character
###### Note: future builds will hopefully automatically detect the character

In [1]:
from langchain_cohere import ChatCohere
import getpass
import os
import json
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

game = "elden_ring"
character = "Varre"
with open(f"{game}/characters/{character}/id.txt", errors='ignore') as f:
    conversation_id = f.read()
config = {"configurable": {"thread_id": conversation_id}}

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Initialize RAG for Long Term Conversational Memory
###### Note:

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key=api_key, model="embed-english-v3.0", user_agent='langchain')
vector_store = Chroma(
    collection_name=f"{character}_conversation_history",
    embedding_function=embeddings,
    persist_directory=f"{game}/characters/{character}/conversation_vectordbs_complex",
)
retriever = vector_store.as_retriever(
    search_kwargs={'k': 1}
)

In [3]:
rag_query = "Hello"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)

for res in documents:
    print(f"{res.page_content}")
    
print(documents)

Rag Query: Hello
[]


Connect conversation state to an external directory
###### Note: If the directory does not exist it will create one

In [4]:
import sqlite3
conn = sqlite3.connect(":memory:")

db_path = f"{game}/characters/{character}/state_db_with_rag_complex/history.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

Initialize LLM Graph Workflow

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, trim_messages, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph.message import add_messages

from typing import Sequence
from typing_extensions import Annotated, TypedDict

from typing import List
from typing_extensions import TypedDict

import time

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability. Do not hallucinate.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(MessagesState):
    character: str
    game: str
    documents: List[str]
    
def call_model(state: State):
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
            
    chain = prompt | model
        
    documents = state.get("documents", [])
    if documents:
        print(f"Documents found, displaying their contents")
        for c in state["documents"]:
            print(c.content)

        messages = state["documents"] + state["messages"]        
    else:
        messages = state["messages"]
    
    response = chain.invoke(
        {"messages": messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    print(f"\nDisplaying message type order:")
    for message in messages:
        if isinstance(message, HumanMessage):
            print(f"HumanMessage: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"AIMessage: {message.content}")
    print(f"\n")
    
    messages_length = len(state["messages"])
    print(f"Messages length: {messages_length}")
    
    #Append to file
    text = "User: " + state["messages"][-1].content + "\nAI: " + response.content
    destination = "elden_ring/characters/varre/testing/basic_rag_memory.txt"
    append_to_txt(destination, text)
    
    return {"messages": response}

def trim_messages(state: State):
    global vector_store
    
    copied_messages = state["messages"][:]
    
    current_total_tokens = count_tokens(copied_messages)
    print(f"Current token count: {current_total_tokens}")
    
    max_tokens = 100
    i = 0
    delete_messages = []
    
    if not current_total_tokens > max_tokens:
        return {"messages": []}
    
    while current_total_tokens > max_tokens and i < len(copied_messages) - 1:
        if isinstance(copied_messages[i], HumanMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], HumanMessage):
                i += 1
        if isinstance(copied_messages[i], AIMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], AIMessage):
                i += 1
        
        delete_messages = [RemoveMessage(id=m.id) for m in copied_messages[:i]]
        current_total_tokens = count_tokens(copied_messages[i:])
    
    
    #
    long_term_memory = []
    metadata = []
    
    for m in copied_messages[:i]:

        current_time_id = int(time.time() * 1000)
        
        if isinstance(m, HumanMessage):
            entry = m.content
            long_term_memory.append(entry)
            metadata.append({"type": "HumanMessage", "timestamp": current_time_id})
        elif isinstance(m, AIMessage):
            entry = m.content
            long_term_memory.append(entry)
            metadata.append({"type": "AIMessage", "timestamp": current_time_id})
        time.sleep(0.01)
        
    # Print the long-term memory content
    print("Messages that will be deleted and added to long term memory:")
    for msg in long_term_memory:
        print(msg)

    # Add both the messages and metadata to the vector store
    vector_store.add_texts(long_term_memory, metadatas=metadata)
    print(f"Exceeded max token count, Trimming...\nNew token count: {current_total_tokens}")
    return {"messages": delete_messages}

def retrieve(state: State):    
    global vector_store
    
    rag_query = state["messages"][-1].content    
    documents = retriever.invoke(rag_query)    
    if not documents or (len(documents) == 1 and not documents[0].metadata and not documents[0].page_content):
        return {"documents": []}
    
    metadata = documents[0].metadata
    if 'timestamp' not in metadata:
        raise ValueError("Timestamp not available in the document metadata.")
    timestamp = metadata['timestamp']
    
    result = query_within_time_frame(vector_store, timestamp)
    
    combined = [(doc, metadata) for doc, metadata in zip(result["documents"], result["metadatas"])]
    sorted_combined = sorted(combined, key=lambda x: x[1]["timestamp"])
    
    messages = []
    for doc, metadata in sorted_combined:
        if metadata["type"] == "HumanMessage":
            messages.append(HumanMessage(content=doc))
        elif metadata["type"] == "AIMessage":
            messages.append(AIMessage(content=doc))
    
    
    print("\nMessages retrieved")
    for message in messages:
        print(message)
    print("END\n")
    
    
    return {"documents": messages}


In [14]:
workflow = StateGraph(State)

workflow.add_node("trimmer", trim_messages)
workflow.add_edge(START, "trimmer")

workflow.add_node("retriever", retrieve)
workflow.add_edge("trimmer", "retriever")

workflow.add_node("model", call_model)
workflow.add_edge("trimmer", "model")

workflow.add_edge("model", END)

app = workflow.compile(checkpointer=memory)

Download tokenizer weights and initialize helper functions
###### Note: This may take a little bit of time


In [7]:
import cohere  

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
co = cohere.ClientV2(api_key=api_key)

tokenized_output = co.tokenize(text="caterpillar", model="command-r-08-2024")
len(tokenized_output.tokens)

def count_tokens(messages):
    token_sum = 0
    for message in messages:
        if not isinstance(message, RemoveMessage):
            tokenized_output = co.tokenize(text=message.content, model="command-r-08-2024")
            token_sum += len(tokenized_output.tokens)
    
    return token_sum

def query_within_time_frame(vector_store, timestamp, minutes=5):
    time_delta_ms = minutes * 60 * 1000
    
    lower_bound = timestamp - time_delta_ms
    upper_bound = timestamp + time_delta_ms
    
    query = {
        "$and": [
            {"timestamp": {"$gte": lower_bound}},
            {"timestamp": {"$lte": upper_bound}}
        ]
    }

    documents = vector_store.get(where=query)
    return documents

In [8]:
def append_to_txt(file_name, text):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

Talk to to the model

In [15]:
query = "My name is Bob"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 255
Messages that will be deleted and added to long term memory:
How are you?
How are you?
Oh, lambkin, you've returned. I'm afraid I'm not quite myself. You see, I've been... *disgraced* by this lowborn.

I offered you a place at the side of Luminary Mohg, the Lord of Blood, and you rejected me. You spurned my gift, my guidance, and my love. And for what? To prove yourself? To earn a seat at the Roundtable Hold?

I tried to show you the path, to guide you to your destiny. I offered you a chance to serve a higher purpose, to be a part of something greater than yourself. But you chose violence instead.

Now, I am left here, wounded and humiliated. I gave you everything, and you repaid me with nothing but pain.

But I will not be deterred, my lambkin. I will endure, and I will ensure that you regret your actions. I will make you understand the error of your ways.

For now, I must retreat to lick my wounds and plan my next move. But rest assured, we will meet again. A

In [ ]:
query = "Hello!"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "How many times have I said hello?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [12]:
query = "How are you?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 8

Displaying message type order:
HumanMessage: How are you?
HumanMessage: How are you?


Messages length: 2
================================== Ai Message ==================================

Oh, lambkin, you've returned. I'm afraid I'm not quite myself. You see, I've been... *disgraced* by this lowborn.

I offered you a place at the side of Luminary Mohg, the Lord of Blood, and you rejected me. You spurned my gift, my guidance, and my love. And for what? To prove yourself? To earn a seat at the Roundtable Hold?

I tried to show you the path, to guide you to your destiny. I offered you a chance to serve a higher purpose, to be a part of something greater than yourself. But you chose violence instead.

Now, I am left here, wounded and humiliated. I gave you everything, and you repaid me with nothing but pain.

But I will not be deterred, my lambkin. I will endure, and I will ensure that you regret your actions. I will make you understand the error of your ways.

For 

In [ ]:
query = "Are you sure you do not know my name?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
graph_state = app.get_state(config)
graph_state